# Partially Signed Bitcoin Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Init Bitcoin RPC client

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
rpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
rpc.getblockcount()

## Mine coins

In [ ]:
import bitcoin.core as bc

In [ ]:
rpc.createwallet('miner')
# rpc.loadwallet('miner')

In [ ]:
rpc.getbalance()

In [ ]:
minerAddr = rpc.getnewaddress("coinbase")
minerAddr

In [ ]:
# coinbase transactions are locked for 100 blocks
_ = list(rpc.generatetoaddress(101, minerAddr))

In [ ]:
rpc.getbalance()

In [ ]:
unspent = rpc.listunspent()
unspent

In [ ]:
# coinbase transaction
r = rpc.getrawtransaction(unspent[0]['outpoint'].hash, verbose=True)
r

In [ ]:
# Coinbase transaction id
bc.b2lx(r['tx'].GetTxid())

In [ ]:
rpc.unloadwallet('miner')

## Generate addresses for Ali and Bob

### Ali

In [ ]:
rpc.createwallet('ali')

In [ ]:
aliFundAddr = rpc.getnewaddress('ali fund')
aliFundSecret = rpc.dumpprivkey(aliFundAddr)
aliFundAddr

In [ ]:
# h = hashlib.sha256(b'correct horse battery staple').digest()
# seckey = CBitcoinSecret.from_secret_bytes(h)

In [ ]:
aliMsigAddr = rpc.getnewaddress('ali multisig key')
aliMsigSecret = rpc.dumpprivkey(aliMsigAddr)

In [ ]:
aliMsigOutAddr = rpc.getnewaddress("ali msig spend")

In [ ]:
rpc.unloadwallet('ali')

### Bob

In [ ]:
rpc.createwallet('bob')

In [ ]:
bobFundAddr = rpc.getnewaddress("bob fund")
bobFundSecret = rpc.dumpprivkey(bobFundAddr)
bobFundAddr

In [ ]:
bobMsigAddr = rpc.getnewaddress('bob multisig key')
bobMsigSecret = rpc.dumpprivkey(bobMsigAddr)

In [ ]:
bobMsigOutAddr = rpc.getnewaddress("bob msig spend")

In [ ]:
rpc.unloadwallet('bob')

## Fund Ali and Bob

In [ ]:
rpc.loadwallet('miner')

In [ ]:
fundTxId = rpc.sendmany("",
                        {aliFundAddr: 10*bc.COIN, bobFundAddr: 11*bc.COIN},
                        comment="Funding Ali and Bob")

In [ ]:
rpc.getrawtransaction(fundTxId, verbose=True)

In [ ]:
# confirm transactions
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
rpc.unloadwallet('miner')

### Check funds

In [ ]:
rpc.loadwallet('ali')
aliUnspent = rpc.listunspent(addrs=[aliFundAddr])
rpc.unloadwallet('ali')
aliUnspent

In [ ]:
rpc.loadwallet('bob')
bobUnspent = rpc.listunspent(addrs=[bobFundAddr])
rpc.unloadwallet('bob')
bobUnspent

## Create 2-of-2 Multisig

In [ ]:
import bitcoin.core.script as bs

### Get inputs
aka ali's and bob's funding tx outputs

In [ ]:
aliTxin = bc.CTxIn(aliUnspent[0]['outpoint'])
aliTxin

In [ ]:
bobTxin = bc.CTxIn(bobUnspent[0]['outpoint'])
bobTxin

### Construct multisig

In [ ]:
# redeem script
msigScript = bs.CScript([
        bs.OP_2,
        aliMsigSecret.pub,
        bobMsigSecret.pub,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

# msigScript = bs.CScript([
#         aliMsigSecret.pub,
#         bs.OP_CHECKSIG
#     ])
msigScript

### Create output

In [ ]:
scriptPubKey = msigScript.to_p2sh_scriptPubKey()
scriptPubKey

In [ ]:
fee = 0.001*bc.COIN
msigFundAmount = aliUnspent[0]['amount'] + bobUnspent[0]['amount'] - fee
msigFundAmount/bc.COIN

In [ ]:
txout = bc.CTxOut(msigFundAmount, scriptPubKey)
txout

### Create transaction

In [ ]:
tx = bc.CMutableTransaction([aliTxin, bobTxin], [txout])
tx

### Sign transaction

In [ ]:
# sign by Ali
sighash = bs.SignatureHash(aliFundAddr.to_redeemScript(),
                           tx,
                           inIdx=0,
                           hashtype=bs.SIGHASH_ALL,
                           amount=10*bc.COIN,
                           sigversion=bs.SIGVERSION_WITNESS_V0)

aliSignature = aliFundSecret.sign(sighash) + bytes([bs.SIGHASH_ALL])
aliWitness = [aliSignature, aliFundSecret.pub]
aliWitness = bc.CTxInWitness(bs.CScriptWitness(aliWitness))
aliWitness

In [ ]:
# sign by Bob
sighash = bs.SignatureHash(bobFundAddr.to_redeemScript(),
                           tx,
                           inIdx=1,
                           hashtype=bs.SIGHASH_ALL,
                           amount=11*bc.COIN,
                           sigversion=bs.SIGVERSION_WITNESS_V0)

bobSignature = bobFundSecret.sign(sighash) + bytes([bs.SIGHASH_ALL])

bobWitness = [bobSignature, bobFundSecret.pub]
bobWitness = bc.CTxInWitness(bs.CScriptWitness(bobWitness))
bobWitness

In [ ]:
tx.wit = bc.CTxWitness([aliWitness, bobWitness])
tx

### Submit transaction

In [ ]:
msigTxid = rpc.sendrawtransaction(tx)

In [ ]:
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
msigTx = rpc.getrawtransaction(msigTxid, verbose=True)
msigTx

## Spend 2-of-2 multisig

In [ ]:
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

### Create spend input

In [ ]:
msigTx['tx'].vout

In [ ]:
spendTxin = bc.CMutableTxIn(bc.COutPoint(msigTxid, 0))
spendTxin

### Create spend output

In [ ]:
msigFundAmount = msigTx['tx'].vout[0].nValue
msigFundAmount / bc.COIN

In [ ]:
aliAmount = (msigFundAmount - fee) / 2
aliAmount / bc.COIN

In [ ]:
bobAmount = (msigFundAmount - fee) / 2
bobAmount / bc.COIN

In [ ]:
assert msigFundAmount - aliAmount - bobAmount > 0

In [ ]:
aliTxout = bc.CTxOut(aliAmount, aliMsigOutAddr.to_scriptPubKey())
bobTxout = bc.CTxOut(bobAmount, bobMsigOutAddr.to_scriptPubKey())

### Create spend transaction

In [ ]:
spendTx = bc.CMutableTransaction([spendTxin], [aliTxout, bobTxout])
spendTx

### Sign spend transaction

In [ ]:
sighash = bs.SignatureHash(msigScript, spendTx, inIdx=0, hashtype=bs.SIGHASH_ALL)
aliSignature = aliMsigSecret.sign(sighash) + bytes([bs.SIGHASH_ALL])
bobSignature = bobMsigSecret.sign(sighash) + bytes([bs.SIGHASH_ALL])

In [ ]:
spendTxin.scriptSig = bs.CScript([bs.OP_0, aliSignature, bobSignature, msigScript])
# spendTxin.scriptSig = bs.CScript([aliSignature, msigScript])

In [ ]:
VerifyScript(spendTxin.scriptSig, msigScript.to_p2sh_scriptPubKey(), spendTx, 0, (SCRIPT_VERIFY_P2SH,))

In [ ]:
spendTxid = rpc.sendrawtransaction(spendTx)

In [ ]:
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
rpc.getrawtransaction(spendTxid, verbose=True)